In [94]:
# Generated code -- CC0 -- No Rights Reserved -- http://www.redblobgames.com/grids/hexagons/

from __future__ import division
from __future__ import print_function
import collections
import math


Point = collections.namedtuple("Point", ["x", "y"])
_Hex = collections.namedtuple("Hex", ["q", "r", "s"])

def Hex(q, r, s):
    assert not (round(q + r + s) != 0), "q + r + s must be 0"
    return _Hex(q, r, s)

def hex_add(a, b):
    return Hex(a.q + b.q, a.r + b.r, a.s + b.s)

def hex_subtract(a, b):
    return Hex(a.q - b.q, a.r - b.r, a.s - b.s)

def hex_scale(a, k):
    return Hex(a.q * k, a.r * k, a.s * k)

def hex_rotate_left(a):
    return Hex(-a.s, -a.q, -a.r)

def hex_rotate_right(a):
    return Hex(-a.r, -a.s, -a.q)

hex_directions = [Hex(1, 0, -1), Hex(1, -1, 0), Hex(0, -1, 1), Hex(-1, 0, 1), Hex(-1, 1, 0), Hex(0, 1, -1)]
def hex_direction(direction):
    return hex_directions[direction]

def hex_neighbor(hex, direction):
    return hex_add(hex, hex_direction(direction))

hex_diagonals = [Hex(2, -1, -1), Hex(1, -2, 1), Hex(-1, -1, 2), Hex(-2, 1, 1), Hex(-1, 2, -1), Hex(1, 1, -2)]
def hex_diagonal_neighbor(hex, direction):
    return hex_add(hex, hex_diagonals[direction])

def hex_length(hex):
    return (abs(hex.q) + abs(hex.r) + abs(hex.s)) // 2

def hex_distance(a, b):
    return hex_length(hex_subtract(a, b))

def hex_round(h):
    qi = int(round(h.q))
    ri = int(round(h.r))
    si = int(round(h.s))
    q_diff = abs(qi - h.q)
    r_diff = abs(ri - h.r)
    s_diff = abs(si - h.s)
    if q_diff > r_diff and q_diff > s_diff:
        qi = -ri - si
    else:
        if r_diff > s_diff:
            ri = -qi - si
        else:
            si = -qi - ri
    return Hex(qi, ri, si)

def hex_lerp(a, b, t):
    return Hex(a.q * (1.0 - t) + b.q * t, a.r * (1.0 - t) + b.r * t, a.s * (1.0 - t) + b.s * t)

def hex_linedraw(a, b):
    N = hex_distance(a, b)
    a_nudge = Hex(a.q + 0.000001, a.r + 0.000001, a.s - 0.000002)
    b_nudge = Hex(b.q + 0.000001, b.r + 0.000001, b.s - 0.000002)
    results = []
    step = 1.0 / max(N, 1)
    for i in range(0, N + 1):
        results.append(hex_round(hex_lerp(a_nudge, b_nudge, step * i)))
    return results




OffsetCoord = collections.namedtuple("OffsetCoord", ["col", "row"])

EVEN = 1
ODD = -1
def qoffset_from_cube(offset, h):
    col = h.q
    row = h.r + (h.q + offset * (h.q & 1)) // 2
    return OffsetCoord(col, row)

def qoffset_to_cube(offset, h):
    q = h.col
    r = h.row - (h.col + offset * (h.col & 1)) // 2
    s = -q - r
    return Hex(q, r, s)

def roffset_from_cube(offset, h):
    col = h.q + (h.r + offset * (h.r & 1)) // 2
    row = h.r
    return OffsetCoord(col, row)

def roffset_to_cube(offset, h):
    q = h.col - (h.row + offset * (h.row & 1)) // 2
    r = h.row
    s = -q - r
    return Hex(q, r, s)




DoubledCoord = collections.namedtuple("DoubledCoord", ["col", "row"])

def qdoubled_from_cube(h):
    col = h.q
    row = 2 * h.r + h.q
    return DoubledCoord(col, row)

def qdoubled_to_cube(h):
    q = h.col
    r = (h.row - h.col) // 2
    s = -q - r
    return Hex(q, r, s)

def rdoubled_from_cube(h):
    col = 2 * h.q + h.r
    row = h.r
    return DoubledCoord(col, row)

def rdoubled_to_cube(h):
    q = (h.col - h.row) // 2
    r = h.row
    s = -q - r
    return Hex(q, r, s)




Orientation = collections.namedtuple("Orientation", ["f0", "f1", "f2", "f3", "b0", "b1", "b2", "b3", "start_angle"])

Layout = collections.namedtuple("Layout", ["orientation", "size", "origin"])

layout_pointy = Orientation(math.sqrt(3.0), math.sqrt(3.0) 
                            / 2.0, 0.0, 3.0 / 2.0, math.sqrt(3.0) / 3.0, -1.0 / 3.0, 0.0, 2.0 / 3.0, 0.5)
layout_flat = Orientation(3.0 / 2.0, 0.0, math.sqrt(3.0) / 2.0, math.sqrt(3.0), 2.0 / 3.0, 0.0, -1.0 
                          / 3.0, math.sqrt(3.0) / 3.0, 0.0)
def hex_to_pixel(layout, h):
    M = layout.orientation
    size = layout.size
    origin = layout.origin
    x = (M.f0 * h.q + M.f1 * h.r) * size.x
    y = (M.f2 * h.q + M.f3 * h.r) * size.y
    return Point(x + origin.x, y + origin.y)

def pixel_to_hex(layout, p):
    M = layout.orientation
    size = layout.size
    origin = layout.origin
    pt = Point((p.x - origin.x) / size.x, (p.y - origin.y) / size.y)

    q = M.b0 * pt.x + M.b1 * pt.y
    r = M.b2 * pt.x + M.b3 * pt.y
    return Hex(q, r, -q - r)

def hex_corner_offset(layout, corner):
    M = layout.orientation
    size = layout.size
    angle = 2.0 * math.pi * (M.start_angle - corner) / 6.0
    return Point(size.x * math.cos(angle), size.y * math.sin(angle))

def polygon_corners(layout, h):
    corners = []
    center = hex_to_pixel(layout, h)
    for i in range(0, 6):
        offset = hex_corner_offset(layout, i)
        corners.append(Point(center.x + offset.x, center.y + offset.y))
    return corners




# Tests

def complain(name):
    print("FAIL {0}".format(name))

def equal_hex(name, a, b):
    if not (a.q == b.q and a.s == b.s and a.r == b.r):
        complain(name)

def equal_offsetcoord(name, a, b):
    if not (a.col == b.col and a.row == b.row):
        complain(name)

def equal_doubledcoord(name, a, b):
    if not (a.col == b.col and a.row == b.row):
        complain(name)

def equal_int(name, a, b):
    if not (a == b):
        complain(name)

def equal_hex_array(name, a, b):
    #비교할 수가 같은지 확인
    equal_int(name, len(a), len(b))
    #a랑 b랑 같은지 확인 
    for i in range(0, len(a)):
        equal_hex(name, a[i], b[i])

def test_hex_arithmetic():
    equal_hex("hex_add", Hex(4, -10, 6), hex_add(Hex(1, -3, 2), Hex(3, -7, 4)))
    equal_hex("hex_subtract", Hex(-2, 4, -2), hex_subtract(Hex(1, -3, 2), Hex(3, -7, 4)))

def test_hex_direction():
    equal_hex("hex_direction", Hex(0, -1, 1), hex_direction(2))

def test_hex_neighbor():
    equal_hex("hex_neighbor", Hex(1, -3, 2), hex_neighbor(Hex(1, -2, 1), 2))

def test_hex_diagonal():
    equal_hex("hex_diagonal", Hex(-1, -1, 2), hex_diagonal_neighbor(Hex(1, -2, 1), 3))

def test_hex_distance():
    equal_int("hex_distance", 7, hex_distance(Hex(3, -7, 4), Hex(0, 0, 0)))

def test_hex_rotate_right():
    equal_hex("hex_rotate_right", hex_rotate_right(Hex(1, -3, 2)), Hex(3, -2, -1))

def test_hex_rotate_left():
    equal_hex("hex_rotate_left", hex_rotate_left(Hex(1, -3, 2)), Hex(-2, -1, 3))

def test_hex_round():
    a = Hex(0.0, 0.0, 0.0)
    b = Hex(1.0, -1.0, 0.0)
    c = Hex(0.0, -1.0, 1.0)
    equal_hex("hex_round 1", Hex(5, -10, 5), hex_round(hex_lerp(Hex(0.0, 0.0, 0.0), Hex(10.0, -20.0, 10.0), 0.5)))
    equal_hex("hex_round 2", hex_round(a), hex_round(hex_lerp(a, b, 0.499)))
    equal_hex("hex_round 3", hex_round(b), hex_round(hex_lerp(a, b, 0.501)))
    equal_hex("hex_round 4", hex_round(a), hex_round(Hex(a.q * 0.4 + b.q * 0.3 + c.q * 0.3, a.r * 0.4 + b.r * 0.3 + c.r * 0.3, a.s * 0.4 + b.s * 0.3 + c.s * 0.3)))
    equal_hex("hex_round 5", hex_round(c), hex_round(Hex(a.q * 0.3 + b.q * 0.3 + c.q * 0.4, a.r * 0.3 + b.r * 0.3 + c.r * 0.4, a.s * 0.3 + b.s * 0.3 + c.s * 0.4)))

def test_hex_linedraw():
    equal_hex_array("hex_linedraw", 
                    [Hex(0, 0, 0), Hex(0, -1, 1), Hex(0, -2, 2), Hex(1, -3, 2), Hex(1, -4, 3), Hex(1, -5, 4)]
                    
                    , hex_linedraw(Hex(0, 0, 0), Hex(1, -5, 4))
                   )

def test_layout():
    h = Hex(3, 4, -7)
    flat = Layout(layout_flat, Point(10.0, 15.0), Point(35.0, 71.0))
    equal_hex("layout", h, hex_round(pixel_to_hex(flat, hex_to_pixel(flat, h))))
   
pointy = Layout(layout_pointy, Point(10.0, 15.0), Point(35.0, 71.0))

    equal_hex("layout", h, hex_to_pixel(pointy, h))

def test_offset_roundtrip():
    a = Hex(3, 4, -7)
    b = OffsetCoord(1, -3)
    equal_hex("conversion_roundtrip even-q", a, qoffset_to_cube(EVEN, qoffset_from_cube(EVEN, a)))
    equal_offsetcoord("conversion_roundtrip even-q", b, qoffset_from_cube(EVEN, qoffset_to_cube(EVEN, b)))
    equal_hex("conversion_roundtrip odd-q", a, qoffset_to_cube(ODD, qoffset_from_cube(ODD, a)))
    equal_offsetcoord("conversion_roundtrip odd-q", b, qoffset_from_cube(ODD, qoffset_to_cube(ODD, b)))
    equal_hex("conversion_roundtrip even-r", a, roffset_to_cube(EVEN, roffset_from_cube(EVEN, a)))
    equal_offsetcoord("conversion_roundtrip even-r", b, roffset_from_cube(EVEN, roffset_to_cube(EVEN, b)))
    equal_hex("conversion_roundtrip odd-r", a, roffset_to_cube(ODD, roffset_from_cube(ODD, a)))
    equal_offsetcoord("conversion_roundtrip odd-r", b, roffset_from_cube(ODD, roffset_to_cube(ODD, b)))

def test_offset_from_cube():
    equal_offsetcoord("offset_from_cube even-q", OffsetCoord(1, 3), qoffset_from_cube(EVEN, Hex(1, 2, -3)))
    equal_offsetcoord("offset_from_cube odd-q", OffsetCoord(1, 2), qoffset_from_cube(ODD, Hex(1, 2, -3)))

def test_offset_to_cube():
    equal_hex("offset_to_cube even-", Hex(1, 2, -3), qoffset_to_cube(EVEN, OffsetCoord(1, 3)))
    equal_hex("offset_to_cube odd-q", Hex(1, 2, -3), qoffset_to_cube(ODD, OffsetCoord(1, 2)))

def test_doubled_roundtrip():
    a = Hex(3, 4, -7)
    b = DoubledCoord(1, -3)
    equal_hex("conversion_roundtrip doubled-q", a, qdoubled_to_cube(qdoubled_from_cube(a)))
    equal_doubledcoord("conversion_roundtrip doubled-q", b, qdoubled_from_cube(qdoubled_to_cube(b)))
    equal_hex("conversion_roundtrip doubled-r", a, rdoubled_to_cube(rdoubled_from_cube(a)))
    equal_doubledcoord("conversion_roundtrip doubled-r", b, rdoubled_from_cube(rdoubled_to_cube(b)))

def test_doubled_from_cube():
    equal_doubledcoord("doubled_from_cube doubled-q", DoubledCoord(1, 5), qdoubled_from_cube(Hex(1, 2, -3)))
    equal_doubledcoord("doubled_from_cube doubled-r", DoubledCoord(4, 2), rdoubled_from_cube(Hex(1, 2, -3)))

def test_doubled_to_cube():
    equal_hex("doubled_to_cube doubled-q", Hex(1, 2, -3), qdoubled_to_cube(DoubledCoord(1, 5)))
    equal_hex("doubled_to_cube doubled-r", Hex(1, 2, -3), rdoubled_to_cube(DoubledCoord(4, 2)))

def test_all():
    test_hex_arithmetic()
    test_hex_direction()
    test_hex_neighbor()
    test_hex_diagonal()
    test_hex_distance()
    test_hex_rotate_right()
    test_hex_rotate_left()
    test_hex_round()
    test_hex_linedraw()
    test_layout()
    test_offset_roundtrip()
    test_offset_from_cube()
    test_offset_to_cube()
    test_doubled_roundtrip()
    test_doubled_from_cube()
    test_doubled_to_cube()





In [95]:
test_all()

In [179]:
#x:경도, y:위도
li = [(127.03401035390544, 37.48454726866469),
  (127.03501627811512, 37.484682633459975),
  (127.03633098534883, 37.4848595343672),
  (127.03681086729952, 37.4849241018911),
  (127.03734233412982, 37.48499560756338),
  (127.03745359832253, 37.48501057544179),
  (127.03757827307057, 37.485027350135276),
  (127.03780125398346, 37.48505734827392),
  (127.03789329585472, 37.48506973576668),
  (127.03802403198914, 37.48508823804522),
  (127.03812310744961, 37.48510225399731),
  (127.03821935610138, 37.48511588331396),
  (127.03949806886966, 37.48529691375585),
  (127.03951889721874, 37.485299862341876),
  (127.04133922435369, 37.48555755279026),
  (127.04175116974153, 37.48561586116136)]

# li = [(958802.5352367972, 1942913.908839113),
#   (958891.5422104027, 1942928.4869633378),
#   (959007.8706680674, 1942947.5397021722),
#   (959050.3316374966, 1942954.4942048327),
#   (959097.3568586954, 1942962.19625241),
#   (959107.2017188016, 1942963.8085054436),
#   (959118.2331662644, 1942965.6154008047),
#   (959137.9629069318, 1942968.8466921893),
#   (959146.1069304948, 1942970.1810576392),
#   (959157.6751783716, 1942972.177038927),
#   (959166.4419098641, 1942973.6890396674),
#   (959174.9585141209, 1942975.1593807805),
#   (959288.1056566194, 1942994.6897795717),
#   (959289.9486524205, 1942995.007898916),
#   (959451.0196732285, 1943022.8113609252),
#   (959487.4703603057, 1943029.1029393973)]

li2 = [ (127.04359887908493, 37.48039823968481),
  (127.04361840519732, 37.4804198117202),
  (127.04372836548231, 37.480558589821804),
  (127.04375134400168, 37.48058729653464),
  (127.04384150520642, 37.48070101573564),
  (127.04404143207256, 37.480926761997495),
  (127.04417489689601, 37.481081569038444),
  (127.04420423261536, 37.481115346004344),
  (127.04424576612153, 37.48114658673793),
  (127.04450626665856, 37.48134105453069),
  (127.04460911965357, 37.48140324023337),
  (127.04464057282112, 37.48142237504143),
  (127.044784605911, 37.48150960240706),
  (127.04485493167373, 37.48155209489671),
  (127.04488250518638, 37.48156813161751),
  (127.044929690156, 37.481595711967586),
  (127.04507565595111, 37.48168068576177),
  (127.04522162224468, 37.48176594769461),
  (127.04536760001272, 37.481850930131095),
  (127.04551356696372, 37.481936182690426),
  (127.04559220554412, 37.481982050397896),
  (127.04565953425161, 37.48202144407797),
  (127.04574259521286, 37.482069841846524),
  (127.04580427016322, 37.482107831997226),
  (127.0458260084379, 37.4821210595587),
  (127.04595112316129, 37.48219843562847),
  (127.04601351077841, 37.48223670471153),
  (127.04607643018741, 37.48227554119174),
  (127.04620154526012, 37.482352637683604),
  (127.04632684149476, 37.48242974298326),
  (127.04645196838798, 37.48250683920334),
  (127.04657726530505, 37.48258422354542),
  (127.04670256231229, 37.48266132844311),
  (127.04682767867129, 37.482738424266124),
  (127.04693831780465, 37.482806524518324),
  (127.04695192419666, 37.482814682359184),
  (127.04695315709365, 37.482815528826336),
  (127.04706537864556, 37.482882493088454),
  (127.0470745739061, 37.48288784149128),
  (127.04714455141627, 37.48292976511105),
  (127.04758637132646, 37.4831937041822),
  (127.0476379698545, 37.483224651466486),
  (127.04788132679991, 37.48336984267792),
  (127.04789422092314, 37.48337772137854),
  (127.04827560311689, 37.48360704707129),
  (127.0484896232875, 37.48373563416382),
  (127.04856808555066, 37.48378290550771),
  (127.04939200692631, 37.484278404479994),
  (127.04940667702496, 37.48428712923567),
  (127.05006535613789, 37.48468330090213),
  (127.05029458452825, 37.4848211767808),
  (127.05048828576055, 37.48493765915765),
  (127.0505377718024, 37.484967768110366),
  (127.05060670148924, 37.4850094107883),
  (127.05120283316916, 37.485370967555845),
  (127.05138981900873, 37.485484360833695),
  (127.05155401321719, 37.48558396013373),
  (127.05157168141278, 37.48559465654079),
  (127.05166889066729, 37.48565373899777),
  (127.05207115526306, 37.48589740543214),
  (127.05220652816551, 37.485977031947556),
  (127.05229224483726, 37.48602738811945),
  (127.05278163972493, 37.48631493843115),
  (127.05280002059008, 37.4863256253339),
  (127.05289368983829, 37.486380491628154),
  (127.05300326888535, 37.48643675647706),
  (127.05308209863387, 37.486484024639935),
  (127.0532056315255, 37.486558023914135),
  (127.05393116782037, 37.48699328406781),
  (127.05410897586037, 37.48709991056489),
  (127.0545831859506, 37.48738436067397),
  (127.05501232194031, 37.48764179926451),
  (127.05506234300964, 37.48767190606511),
  (127.05513021402672, 37.48771242031249),
  (127.05528628408457, 37.487806107507936),
  (127.05592134141989, 37.48817747950802),
  (127.0561044472288, 37.48828466816969),
  (127.05611610976612, 37.488291420284376),
  (127.05619352859696, 37.48833671381628),
  (127.0562195121117, 37.48835191067686),
  (127.05631336730286, 37.4884067741536),
  (127.05674445596544, 37.48865885355646),
  (127.05713823580014, 37.488852663230205),
  (127.05719249621788, 37.48887938837503),
  (127.05730772316781, 37.48893395266357),
  (127.05732468982795, 37.488941828403924),
  (127.05757548202313, 37.48906081291468),
  (127.05786974382401, 37.489184839787185),
  (127.05789324766498, 37.48919468551808),
  (127.05821806633392, 37.489323481247176),
  (127.05845576201519, 37.489405583213546),
  (127.05847290832739, 37.4894114945211),
  (127.05852716341383, 37.48943004698019),
  (127.05911757764156, 37.489613060111864),
  (127.05958673942266, 37.48972883423619),
  (127.05969807004054, 37.48975636809288),
  (127.05985816712365, 37.48979570790717),
  (127.06040860041506, 37.489901302352514),
  (127.06071765145737, 37.48995690179497),
  (127.06096150665927, 37.4900007031157),
  (127.06108307818138, 37.49002259926169),
  (127.06140256033324, 37.49008410221284),
  (127.0617739859399, 37.49015571407115),
  (127.06181784061708, 37.49016148512793),
  (127.06184855473201, 37.49016553295891),
  (127.06223251527796, 37.49021601752958),
  (127.06226026634678, 37.49021938201598),
  (127.0625109894923, 37.490252472723384),
  (127.06289283482761, 37.4903009830488),
  (127.06297941352351, 37.49031220054326),
  (127.06319198193532, 37.49034024602628),
  (127.06334817841588, 37.490362689390494),
  (127.06357241962479, 37.49039467445993),
  (127.06390903707887, 37.490457287877376),
  (127.06407938829601, 37.49049013835977),
  (127.06411030797466, 37.490496311853846),
  (127.06421244186016, 37.49051597158329),
  (127.06445046862954, 37.4905625799105),
  (127.06460703465847, 37.49059319348319),
  (127.06499544093835, 37.49066787656779),
  (127.06506965263638, 37.49068219861363),
  (127.06518893152341, 37.4907046593948),
  (127.06524123690929, 37.49071448813592),
  (127.06538578085346, 37.49074171939898),
  (127.0654916358019, 37.49076165532003),
  (127.06555012716956, 37.490772318504675),
  (127.06589028574957, 37.4908332304729),
  (127.06593710598038, 37.49084165633412),
  (127.06637216217185, 37.490920814444095),
  (127.06662468747402, 37.49098008749762),
  (127.06664802023569, 37.49098627351754),
  (127.0668063625449, 37.49102898362384),
  (127.06704477603135, 37.4911003640054),
  (127.06732771154283, 37.491201848553025),
  (127.06750322219534, 37.49127327203807),
  (127.06761280916251, 37.491317981470814),
  (127.06768120170241, 37.49134863124866),
  (127.06782985331962, 37.49141527596502),
  (127.06789225177914, 37.49144452342435),
  (127.06805220504398, 37.491516792759526),
  (127.06831841772721, 37.4916543333487),
  (127.06859771610713, 37.49181916620185),
  (127.0686274165215, 37.49183914252237),
  (127.06869124145183, 37.491882183055004),
  (127.06879590050012, 37.49195279793264),
  (127.06893715528842, 37.492052953644006),
  (127.06894670553402, 37.4920602643125),
  (127.06933620249684, 37.49235258700456),
  (127.0694580184872, 37.492450221433074),
  (127.06964863974856, 37.49261960773685),
  (127.0697340543696, 37.49270206299637),
  (127.06976606064339, 37.49273273256277),
  (127.06981770336068, 37.49278253701427),
  (127.06984175341582, 37.49280561573129),
  (127.07011038763358, 37.493084769979404),
  (127.07017104499775, 37.49314893097382),
  (127.07046711143778, 37.493461567615384),
  (127.07067579907088, 37.49368303748174),
  (127.07076458514112, 37.493777031811206),
  (127.07085071223622, 37.493869045433804),
  (127.07089793557486, 37.49391941962599),
  (127.07094993536197, 37.49397457529942),
  (127.07100492949004, 37.49402719737964),
  (127.07118142484315, 37.49419828334832)]

# for i in range(0, len(li2)):
#     li.append(li2[i])

data = li

In [185]:
data = {}

for i in range(0,len(li)):
        data[i] = Point(li[i][0] * 100000, li[i][1] * 100000)


In [211]:
#5로하면 30meter 구분, 10으로하면 같은점

pointy = Layout(layout_pointy, Point(8,8),  data[0])
hexli = []
for i in range(0, len(data) ) :     
    hexli.append( pixel_to_hex(pointy, data[i] ) )

hexli

[Hex(q=0.0, r=0.0, s=-0.0),
 Hex(q=6.695612685152127, r=1.128039960671837, s=-7.823652645823964),
 Hex(q=15.44660776273375, r=2.602214187577677, s=-18.048821950311428),
 Hex(q=18.64082608091533, r=3.1402768867556006, s=-21.78110296767093),
 Hex(q=22.178417248860743, r=3.7361574890480065, s=-25.91457473790875),
 Hex(q=22.91903123404657, r=3.8608898091673227, s=-26.779921043213893),
 Hex(q=23.74889916958469, r=4.0006789215452345, s=-27.749578091129923),
 Hex(q=25.23313305124109, r=4.250663410212534, s=-29.483796461453622),
 Hex(q=25.845773487496338, r=4.353892516538811, s=-30.199666004035148),
 Hex(q=26.712187440255153, r=4.5080781710955, s=-31.220265611350655),
 Hex(q=27.368803186918576, r=4.624877771828324, s=-31.9936809587469),
 Hex(q=28.006629180241976, r=4.738455410542277, s=-32.74508459078425),
 Hex(q=36.480650184595, r=6.247042426335004, s=-42.727692610930006),
 Hex(q=36.61868007140218, r=6.2716139765301095, s=-42.890294047932294),
 Hex(q=48.68204938702411, r=8.419034379728448, s=

In [212]:
r_h = []
for i in range(0, len(hexli) ) :
    r_h.append(hex_round(hexli[i]))
    print( "\n", hexli[i] , " : " , r_h[i] )


 Hex(q=0.0, r=0.0, s=-0.0)  :  Hex(q=0, r=0, s=0)

 Hex(q=6.695612685152127, r=1.128039960671837, s=-7.823652645823964)  :  Hex(q=7, r=1, s=-8)

 Hex(q=15.44660776273375, r=2.602214187577677, s=-18.048821950311428)  :  Hex(q=15, r=3, s=-18)

 Hex(q=18.64082608091533, r=3.1402768867556006, s=-21.78110296767093)  :  Hex(q=19, r=3, s=-22)

 Hex(q=22.178417248860743, r=3.7361574890480065, s=-25.91457473790875)  :  Hex(q=22, r=4, s=-26)

 Hex(q=22.91903123404657, r=3.8608898091673227, s=-26.779921043213893)  :  Hex(q=23, r=4, s=-27)

 Hex(q=23.74889916958469, r=4.0006789215452345, s=-27.749578091129923)  :  Hex(q=24, r=4, s=-28)

 Hex(q=25.23313305124109, r=4.250663410212534, s=-29.483796461453622)  :  Hex(q=25, r=4, s=-29)

 Hex(q=25.845773487496338, r=4.353892516538811, s=-30.199666004035148)  :  Hex(q=26, r=4, s=-30)

 Hex(q=26.712187440255153, r=4.5080781710955, s=-31.220265611350655)  :  Hex(q=27, r=4, s=-31)

 Hex(q=27.368803186918576, r=4.624877771828324, s=-31.9936809587469)  :  He

In [218]:
print( "0 : " , "Hex(1, -2, 1)" )
print( "1 : " , hex_neighbor(Hex(1, -2, 1),1) )
print("2 : " , hex_neighbor(Hex(1, -2, 1),2) )
print( "3 : " ,hex_neighbor(Hex(1, -2, 1),3) )
print( "4 : " ,hex_neighbor(Hex(1, -2, 1),4) )
print("5 : " , hex_neighbor(Hex(1, -2, 1),5) )

0 :  Hex(1, -2, 1)
1 :  Hex(q=2, r=-3, s=1)
2 :  Hex(q=1, r=-3, s=2)
3 :  Hex(q=0, r=-2, s=2)
4 :  Hex(q=0, r=-1, s=1)
5 :  Hex(q=1, r=-1, s=0)
